UKF（无迹卡尔曼滤波器）和EKF（扩展卡尔曼滤波器）都是用于处理非线性系统的滤波器，但它们在处理非线性系统时的方法和性能上有显著的不同：

### 1. 非线性处理方法

**EKF（扩展卡尔曼滤波器）**：
- EKF通过对系统的非线性函数（状态转移函数和观测函数）进行线性化来处理非线性问题。具体来说，它使用泰勒级数展开的一阶导数（即雅可比矩阵）来近似非线性函数。
- 这种方法在非线性程度较低时效果较好，但在高度非线性的情况下，由于只保留了一阶项，可能会忽略重要的高阶信息，导致估计精度下降。

**UKF（无迹卡尔曼滤波器）**：
- UKF使用无迹变换（Unscented Transform）来捕捉非线性系统的概率分布。它通过选择一组特定的点（称为Sigma点），这些点围绕当前状态的概率分布对称分布，并通过这些点的传播来近似非线性变换后的概率分布。
- UKF不需要计算雅可比矩阵，而是直接使用Sigma点来捕捉非线性变换的均值和协方差，因此能够更好地处理高度非线性的系统。

### 2. 计算复杂度

**EKF**：
- EKF的计算复杂度相对较低，因为它只涉及到雅可比矩阵的计算和线性代数运算。
- 但是，对于高维系统，雅可比矩阵的计算和存储可能会变得复杂和耗时。

**UKF**：
- UKF的计算复杂度较高，因为它需要生成和处理多个Sigma点，每个Sigma点都需要通过非线性函数传播。
- 尽管计算量更大，但UKF通常能提供更准确的结果，尤其是在处理高度非线性系统时。

### 3. 精度和稳定性

**EKF**：
- EKF在系统非线性较小的情况下表现良好，但在非线性较大时，由于线性化引入的误差，可能会导致滤波器发散或精度下降。

**UKF**：
- UKF在处理高度非线性系统时通常能提供更高的精度和更好的稳定性，因为它通过Sigma点更全面地捕捉了非线性变换的影响。

### 4. 实现复杂度

**EKF**：
- EKF的实现相对简单，因为它只需要计算雅可比矩阵，这对于许多工程师和研究人员来说是一个熟悉的概念。

**UKF**：
- UKF的实现更复杂，需要正确地生成和处理Sigma点，以及计算它们的均值和协方差，这可能需要更多的数学背景和编程工作。

总结来说，UKF和EKF在处理非线性系统时的主要区别在于它们处理非线性的方式、计算复杂度、精度和稳定性，以及实现的复杂度。在选择滤波器时，需要根据具体的应用场景和系统特性来决定使用哪种方法。


### EKF优势情况
**环境**：无人机姿态估计中，系统非线性程度不高，计算资源有限，需要实时应用。

**Python代码示例**：

In [3]:
import numpy as np

In [4]:
class EKF:
    def __init__(self, x, P, F, H, Q, R):
        self.x = x  # 状态估计
        self.P = P  # 状态协方差
        self.F = F  # 状态转移矩阵
        self.H = H  # 观测矩阵
        self.Q = Q  # 过程噪声协方差
        self.R = R  # 观测噪声协方差

    def predict(self):
        # 进行状态预测
        self.x = self.F.dot(self.x)  # 更新状态估计
        self.P = self.F.dot(self.P).dot(self.F.T) + self.Q  # 更新协方差矩阵

    def update(self, z):
        # 进行更新
        # 计算预测的观测值
        z_pred = self.H.dot(self.x)  # 使用 H 矩阵来映射状态到观测空间
        y = z - z_pred  # 观测残差
        H_t = self.H.T  # 观测矩阵的转置

        S = self.H.dot(self.P).dot(H_t) + self.R  # 残差协方差
        K = self.P.dot(H_t).dot(np.linalg.inv(S))  # 卡尔曼增益

        # 更新状态估计和协方差矩阵
        self.x = self.x + K.dot(y)
        self.P = (np.eye(len(self.x)) - K.dot(self.H)).dot(self.P)


In [5]:
# 假设的状态转移和观测模型
def state_transition(x):
    return np.array([1, 0, 0, 1]).dot(x)  # 简单的线性状态转移

def observation(x):
    return np.array([1, 0]).dot(x)  # 简单的线性观测模型

# 初始化EKF
x = np.array([0, 1, 0, 1])  # 初始状态
P = np.eye(4)  # 初始状态协方差
F = np.array([[1, 0, 1, 0], [0, 1, 0, 1], [0, 0, 1, 0], [0, 0, 0, 1]])  # 状态转移矩阵
H = np.array([[1, 0, 0, 0], [0, 1, 0, 0]])  # 观测矩阵
Q = np.eye(4) * 0.1  # 过程噪声协方差
R = np.eye(2) * 0.1  # 观测噪声协方差

ekf = EKF(x, P, F, H, Q, R)

# 模拟无人机姿态估计过程中的状态和观测
true_state = np.array([1, 1, 0.5, 0.5])
measurements = np.array([[1, 1], [0.9, 1.1]])  # 包含噪声的观测

# EKF滤波过程
for z in measurements:
    ekf.predict()
    ekf.update(z)
    print("Estimated State:", ekf.x)  # 打印估计的状态

Estimated State: [0.95454545 1.04545455 0.45454545 0.54545455]
Estimated State: [0.94933921 1.14757709 0.11365639 0.21674009]


### UKF优势情况
**环境**：机器人定位与地图构建（SLAM）中，系统非线性程度高，需要高精度估计。

**Python代码示例**：
```python

In [6]:
# 初始化UKF
ukf = UKF(dim_x=2, dim_z=2, fx=fx, hx=hx, dt=1, alpha=0.001, beta=2, kappa=-1)

# 模拟SLAM环境中的状态和观测
true_state = np.array([1.0, 1.0])
measurements = np.array([[1.1, 1.1], [0.9, 0.9]])  # 包含噪声的观测

# UKF滤波过程
for z in measurements:
    ukf.predict()
    ukf.update(z)
    print("Estimated State:", ukf.state)  # 打印估计的状态

预测过程中发生错误: 'UKF' object has no attribute '_generate_sigma_points'
Estimated State: [1.1 1.1]
预测过程中发生错误: 'UKF' object has no attribute '_generate_sigma_points'
Estimated State: [0.9 0.9]


In [7]:
# 初始化UKF
ukf = UKF(dim_x=2, dim_z=2, fx=fx, hx=hx, dt=1, alpha=0.001, beta=2, kappa=-1)

# 模拟SLAM环境中的状态和观测
true_state = np.array([1.0, 1.0])
measurements = np.array([[1.1, 1.1], [0.9, 0.9]])  # 包含噪声的观测

# UKF滤波过程
for z in measurements:
    ukf.predict()
    ukf.update(z)
    print("Estimated State:", ukf.state)  # 打印估计的状态

预测过程中发生错误: 'UKF' object has no attribute '_generate_sigma_points'
Estimated State: [1.1 1.1]
预测过程中发生错误: 'UKF' object has no attribute '_generate_sigma_points'
Estimated State: [0.9 0.9]


In [15]:
# 定义UKF类
class UKF:
    def __init__(self, dim_x, dim_z, fx, hx, dt, alpha, beta, kappa):
        self.dim_x = dim_x  # 状态维度
        self.dim_z = dim_z  # 观测维度
        self.fx = fx  # 状态转移函数
        self.hx = hx  # 观测函数
        self.dt = dt  # 时间步长
        self.alpha = alpha  # 调整参数
        self.beta = beta  # 先验分布的参数
        self.kappa = kappa  # 二次权重参数
        self.state = np.zeros(dim_x)  # 初始化状态
        self.P = np.eye(dim_x)  # 初始化状态协方差

        self.lambd = alpha ** 2 * (dim_x + kappa) - dim_x
        self.gamma = np.sqrt(dim_x + self.lambd)

    def _generate_sigma_points(self):
        sigma_points = np.zeros((self.dim_x, 2 * self.dim_x + 1))
        sigma_points[:, 0] = self.state  # 第一列为当前状态

        sqrt_P = np.linalg.cholesky(self.P)

        for i in range(self.dim_x):
            sigma_points[:, i + 1] = self.state + self.gamma * sqrt_P[:, i]
            sigma_points[:, i + 1 + self.dim_x] = self.state - self.gamma * sqrt_P[:, i]

        return sigma_points

    def predict(self):
        sigma_points = self._generate_sigma_points()
        predicted_sigma_points = np.zeros_like(sigma_points)

        # 使用 sigma 点进行状态预测
        for i in range(sigma_points.shape[1]):
            predicted_sigma_points[:, i] = self.fx(sigma_points[:, i], self.dt)

        # 计算状态均值和协方差
        self.state = np.mean(predicted_sigma_points, axis=1)
        # 可以在这里添加更新协方差矩阵的逻辑

    def update(self, z):
        z_pred = self.hx(self.state)  # 预测观测
        y = z - z_pred  # 观测残差

        # 计算残差协方差
        S = self.H.dot(self.P).dot(self.H.T) + self.R  # 需要添加观测协方差
        K = self.P.dot(self.H.T).dot(np.linalg.inv(S))  # 卡尔曼增益

        # 更新状态
        self.state = self.state + K.dot(y)
        # 更新协方差矩阵
        self.P = (np.eye(self.dim_x) - K.dot(self.H)).dot(self.P)




# 假设的状态转移和观测模型
def fx(x, dt):
    return x + dt * x  # 简单的线性增长模型

def hx(x):
    return x  # 观测等于状态




In [16]:
# 初始化UKF
ukf = UKF(dim_x=2, dim_z=2, fx=fx, hx=hx, dt=1, alpha=0.001, beta=2, kappa=-1)

# 模拟SLAM环境中的状态和观测
true_state = np.array([1.0, 1.0])
measurements = np.array([[1.1, 1.1], [0.9, 0.9]])  # 包含噪声的观测

# UKF滤波过程
for z in measurements:
    ukf.predict()  # 预测步骤
    ukf.update(z)  # 更新步骤
    print("Estimated State:", ukf.state)  # 打印估计的状态

AttributeError: 'UKF' object has no attribute 'H'

```


以上代码示例分别展示了UKF和EKF在各自优势场景下的应用。UKF适用于非线性程度高的场景，如SLAM，而EKF适用于非线性程度不高且需要实时性的场景，如无人机姿态估计。请注意，这些代码示例需要根据具体的应用场景进行调整和优化。


In [ ]:
import numpy as np

# 状态转移函数
def fx(x, dt):
    # 假设状态保持不变
    return x  # 简单示例，实际中根据需求调整

# 观测函数
def hx(x):
    return x  # 观测等于状态

class UKF:
    def __init__(self, dim_x, dim_z, fx, hx, dt, alpha, beta, kappa):
        if dim_x <= 0 or dim_z <= 0:
            raise ValueError("状态维度和观测维度必须为正数")
        
        self.dim_x = dim_x
        self.dim_z = dim_z
        self.fx = fx  # 状态转移函数
        self.hx = hx  # 观测函数
        self.dt = dt  # 时间步长
        self.alpha = alpha  # 调整参数
        self.beta = beta  # 先验分布的参数
        self.kappa = kappa  # 二次权重参数
        self.state = np.zeros(dim_x)  # 初始化状态
        self.P = np.eye(dim_x) * 0.1  # 初始化状态协方差
        # 添加 H 矩阵
        self.H = np.eye(dim_z)
        # 计算lambda和gamma
        self.lambd = alpha ** 2 * (dim_x + kappa) - dim_x
        self.gamma = np.sqrt(dim_x + self.lambd)

    def _generate_sigma_points(self):
        sigma_points = np.zeros((self.dim_x, 2 * self.dim_x + 1))
        sigma_points[:, 0] = self.state

        sqrt_P = np.linalg.cholesky(self.P)

        for i in range(self.dim_x):
            sigma_points[:, i + 1] = self.state + self.gamma * sqrt_P[:, i]
            sigma_points[:, i + 1 + self.dim_x] = self.state - self.gamma * sqrt_P[:, i]

        return sigma_points

    def predict(self):
        sigma_points = self._generate_sigma_points()
        predicted_sigma_points = np.array([self.fx(sigma_points[:, i], self.dt) for i in range(sigma_points.shape[1])])

        self.state = np.mean(predicted_sigma_points, axis=1)
        # 协方差更新逻辑可以在此处实现

    def update(self, z):
        if len(z) != self.dim_z:
            raise ValueError("观测维度应与dim_z一致")

        z_pred = self.hx(self.state)
        y = z - z_pred 
        
        # 计算观测协方差，添加正则化
        S = self.H.dot(self.P).dot(self.H.T) + 1e-5 * np.eye(self.dim_z)  # 通过增加小常数来确保正定性
        K = self.P.dot(self.H.T).dot(np.linalg.inv(S))

        # 更新状态和协方差
        self.state += K.dot(y)
        self.P -= K.dot(S).dot(K.T)

        # 确保协方差矩阵P为正定
        self.P = (self.P + self.P.T) / 2  # 保持P对称
        self.P += 1e-6 * np.eye(self.dim_x)  # 添加小的正则化项

# 初始化UKF
ukf = UKF(dim_x=2, dim_z=2, fx=fx, hx=hx, dt=1, alpha=0.001, beta=2, kappa=-1)

# 模拟状态和观测
true_state = np.array([1.0, 1.0])
measurements = np.array([[1.1, 1.1], [0.9, 0.9]])

# UKF滤波过程
for z in measurements:
    ukf.predict()
    ukf.update(z)
    print("Estimated State:", ukf.state)  # 打印估计的状态

